In [ ]:
import os

import torch
import matplotlib.pyplot as plt
import networkx as nx
import collections

from copy import deepcopy

# local
import optimization_utils as ou

import sys
if '..' not in sys.path:
    sys.path.insert(0, '..')


from datasets.import_dataset import import_dataset
from trainer import Trainer
from utils.plotting import *
from utils import utils
from utils import utils_pyg as up
import datasets.simulations as sim
import utils.link_prediction as lp




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device = {device}')


%load_ext autoreload
%autoreload 2

Parameters:
- There is a global parameter config for each model and one for every dataset in experiments/hypers.
- The individual dataset parameter config's values replace the appropriate fields in the global config. if a field is not given, the parameter is the same as in the global config.
- The user can provide triplets according to the fields of config dict and they will replace the global or dataset parameters. The config triplets are given as lists to crossval over.

Training goes as follows: 
1. omit both validation and test sets.
2. collect scores from different test sets where the validation set is sampled randomly at every iteration.
3. take the maximal validation parameters and train the model on the combined training and validation set.
4. report the test score 

# TEXAS

In [ ]:
#todo: the saving is ok. you now need to run a lot of experiments on the server

In [ ]:
#DDI ieclam h@20
#DECIDE ON A MODEL AND DATASET
model_name = 'pieclam'
ds_name = 'texas'

# SET RANGE PARAMETERS
use_global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.0
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''
dim_feats = [120, 170]
n_iters_feats = [1000]
n_iters_prior = [1000]
n_back_forth = [2] 
lr_feats = [3e-6, 5*1e-6, 1e-5]
l1_regs = [0.1, 0.5, 0.7]
s_regs = [0.0, 0.1]
range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
    ['prior_opt','n_iter', n_iters_prior],
    ['back_forth','n_back_forth', n_back_forth],
    ]

# if model_name in ['pclam', 'pieclam']:
#     range_triplets += [
#         ['back_forth', 'first_func_in_fit', first_funcs_in_fit],
#         ['prior_opt','n_iter', n_iters_prior],
#         ['prior_opt','lr', lr_prior],
#         ['prior_opt','noise_amp', noise_amps],
#         ['back_forth','n_back_forth', n_back_forth]
#     ]

'''another method is to give a list of perturbations to the base config'''
#! when i do the deltas the parameters arent saved
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)
#todo: check that the omitted_test set is the real omitted set

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=use_global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    metric='auc',
    n_reps=n_reps,
    acc_every=100,
    test_p=0.1,
    val_p=0.1,
    test_dyads_path=None,
    device=device,
    plot_every= 7000
    )
#test that texas still works


# OGB Dataset

## IeClam

In [ ]:
#DDI ieclam h@20
#DECIDE ON A MODEL AND DATASET
model_name = 'ieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS
use_global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.0
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''
dim_feats = [120, 170]
n_iters_feats = [14000] 
lr_feats = [3e-6, 5*1e-6, 1e-5]
l1_regs = [0.1, 0.5, 0.7]
s_regs = [0.0, 0.1]
range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
    ]

# if model_name in ['pclam', 'pieclam']:
#     range_triplets += [
#         ['back_forth', 'first_func_in_fit', first_funcs_in_fit],
#         ['prior_opt','n_iter', n_iters_prior],
#         ['prior_opt','lr', lr_prior],
#         ['prior_opt','noise_amp', noise_amps],
#         ['back_forth','n_back_forth', n_back_forth]
#     ]

'''another method is to give a list of perturbations to the base config'''
#! when i do the deltas the parameters arent saved
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=use_global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    metric='ogb_hAk',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    plot_every= 7000
    )
#test that texas still works


In [ ]:
#DDI ieclam h@20
#DECIDE ON A MODEL AND DATASET
model_name = 'ieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS
use_global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.0
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''
dim_feats = [120, 170]
n_iters_feats = [14000] 
lr_feats = [3e-6, 5*1e-6, 1e-5]
l1_regs = [0.1, 0.5, 0.7]
s_regs = [0.0, 0.1]
range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
    ]

# if model_name in ['pclam', 'pieclam']:
#     range_triplets += [
#         ['back_forth', 'first_func_in_fit', first_funcs_in_fit],
#         ['prior_opt','n_iter', n_iters_prior],
#         ['prior_opt','lr', lr_prior],
#         ['prior_opt','noise_amp', noise_amps],
#         ['back_forth','n_back_forth', n_back_forth]
#     ]

'''another method is to give a list of perturbations to the base config'''
#! when i do the deltas the parameters arent saved
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=use_global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    metric='ogb_hAk',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    plot_every= 7000
    )
#test that texas still works


In [ ]:
#DDI ieclam h@20
#DECIDE ON A MODEL AND DATASET
model_name = 'ieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS
use_global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.0
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''
dim_feats = [120, 170]
n_iters_feats = [14000] 
lr_feats = [3e-6, 5*1e-6, 1e-5]
l1_regs = [0.1, 0.5, 0.7]
s_regs = [0.0, 0.1]
range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
    ]

# if model_name in ['pclam', 'pieclam']:
#     range_triplets += [
#         ['back_forth', 'first_func_in_fit', first_funcs_in_fit],
#         ['prior_opt','n_iter', n_iters_prior],
#         ['prior_opt','lr', lr_prior],
#         ['prior_opt','noise_amp', noise_amps],
#         ['back_forth','n_back_forth', n_back_forth]
#     ]

'''another method is to give a list of perturbations to the base config'''
#! when i do the deltas the parameters arent saved
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=use_global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    metric='ogb_hAk',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    plot_every= 7000
    )
#test that texas still works


In [ ]:
#DDI ieclam h@20
#DECIDE ON A MODEL AND DATASET
model_name = 'ieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS
use_global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.0
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''
dim_feats = [120, 170]
n_iters_feats = [14000] 
lr_feats = [3e-6, 5*1e-6, 1e-5]
l1_regs = [0.1, 0.5, 0.7]
s_regs = [0.0, 0.1]
range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
    ]

# if model_name in ['pclam', 'pieclam']:
#     range_triplets += [
#         ['back_forth', 'first_func_in_fit', first_funcs_in_fit],
#         ['prior_opt','n_iter', n_iters_prior],
#         ['prior_opt','lr', lr_prior],
#         ['prior_opt','noise_amp', noise_amps],
#         ['back_forth','n_back_forth', n_back_forth]
#     ]

'''another method is to give a list of perturbations to the base config'''
#! when i do the deltas the parameters arent saved
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=use_global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    metric='ogb_hAk',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    plot_every= 7000
    )
#test that texas still works


In [ ]:
#DDI ieclam h@20
#DECIDE ON A MODEL AND DATASET
model_name = 'ieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS
use_global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.0
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''
dim_feats = [120, 170]
n_iters_feats = [14000] 
lr_feats = [3e-6, 5*1e-6, 1e-5]
l1_regs = [0.1, 0.5, 0.7]
s_regs = [0.0, 0.1]
range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
    ]

# if model_name in ['pclam', 'pieclam']:
#     range_triplets += [
#         ['back_forth', 'first_func_in_fit', first_funcs_in_fit],
#         ['prior_opt','n_iter', n_iters_prior],
#         ['prior_opt','lr', lr_prior],
#         ['prior_opt','noise_amp', noise_amps],
#         ['back_forth','n_back_forth', n_back_forth]
#     ]

'''another method is to give a list of perturbations to the base config'''
#! when i do the deltas the parameters arent saved
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=use_global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    metric='ogb_hAk',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    plot_every= 7000
    )
#test that texas still works


In [ ]:
#DDI ieclam h@20
#DECIDE ON A MODEL AND DATASET
model_name = 'ieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS
use_global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.0
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''
dim_feats = [120, 170]
n_iters_feats = [14000] 
lr_feats = [3e-6, 5*1e-6, 1e-5]
l1_regs = [0.1, 0.5, 0.7]
s_regs = [0.0, 0.1]
range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
    ]

# if model_name in ['pclam', 'pieclam']:
#     range_triplets += [
#         ['back_forth', 'first_func_in_fit', first_funcs_in_fit],
#         ['prior_opt','n_iter', n_iters_prior],
#         ['prior_opt','lr', lr_prior],
#         ['prior_opt','noise_amp', noise_amps],
#         ['back_forth','n_back_forth', n_back_forth]
#     ]

'''another method is to give a list of perturbations to the base config'''
#! when i do the deltas the parameters arent saved
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=use_global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    metric='ogb_hAk',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    plot_every= 7000
    )
#test that texas still works


In [ ]:
#DDI only on test dataset
#DECIDE ON A MODEL AND DATASET
model_name = 'ieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS

densify = False
attr_opt = False
n_reps = 10

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=[],
    use_global_config_base=False,
    densify=densify,
    attr_opt=attr_opt,
    metric='ogb_hAk',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    test_only=True,
    plot_every= -1
    #! add calculation type and finish the hAk function
    )
#test that texas still works
#todo: i would try ieclam with 1e-5


## bigclam

In [ ]:
#DDI bigclam
#DECIDE ON A MODEL AND DATASET
model_name = 'bigclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS
use_global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.0
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''
dim_feats = [50, 90, 120]
n_iters_feats = [14000] 
lr_feats = [3*1e-6, 6*1e-6, 1e-5]
# l1_regs = [1.0, 0.1]
range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
    ]

'''another method is to give a list of perturbations to the base config'''
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)
#todo: make metric a list

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=use_global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    metric='ogb_hAk',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    plot_every= 500
    #! add calculation type and finish the hAk function
    )
#test that texas still works

## PieClam

In [ ]:
#DDI pieclam hits@20
#DECIDE ON A MODEL AND DATASET
model_name = 'pieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS
densify = False
attr_opt = False
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''

range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', [170, 120]],
    ['feat_opt','n_iter', [2000, 1500]],
    ['feat_opt','lr', [1e-5, 2e-5]],
    ['prior_opt', 'lr', [1e-6, 5e-66]],
    ['prior_opt', 'n_iter', [1000, 2000]],
    ['prior_opt', 'noise_amp', [0.05, 0.03]],
    ['back_forth', 'scheduler_step_size', [4]],
    ['back_forth', 'scheduler_gamma', [0.8]],
    ['back_forth', 'n_back_forth', [7]]
    ][::-1] 

'''another method is to give a list of perturbations to the base config'''
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)
#todo: make metric a list

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=True,
    densify=densify,
    attr_opt=attr_opt,
    metric='ogb_hAk',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    plot_every= 2
    #! add calculation type and finish the hAk function
    )
#test that texas still works

# Auc datasets

In [ ]:
#DDI ieclam h@20
#DECIDE ON A MODEL AND DATASET
model_name = 'ieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS
use_global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.0
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''
dim_feats = [120, 170]
n_iters_feats = [14000] 
lr_feats = [3e-6, 5*1e-6, 1e-5]
l1_regs = [0.1, 0.5, 0.7]
s_regs = [0.0, 0.1]
range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
    ]

# if model_name in ['pclam', 'pieclam']:
#     range_triplets += [
#         ['back_forth', 'first_func_in_fit', first_funcs_in_fit],
#         ['prior_opt','n_iter', n_iters_prior],
#         ['prior_opt','lr', lr_prior],
#         ['prior_opt','noise_amp', noise_amps],
#         ['back_forth','n_back_forth', n_back_forth]
#     ]

'''another method is to give a list of perturbations to the base config'''
#! when i do the deltas the parameters arent saved
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=use_global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    metric='auc',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    plot_every= 7000
    )
#test that texas still works

In [ ]:
#DDI pieclam auc
#DECIDE ON A MODEL AND DATASET
model_name = 'pieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS
densify = False
attr_opt = False
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''
dim_feats = [170, 150, 190]
n_iters_feats = [1000, 2000, 3000] 
lr_feats = [1e-5, 5e-6]
# l1_regs = [1.0, 0.1]

lr_priors = [1e-6, 5e-6]
n_iters_prior = [1000, 2000]
noise_amps = [0.05]

range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
    ['prior_opt', 'lr', lr_feats],
    ['prior_opt', 'n_iters', n_iters_prior],
    ['prior_opt', 'noise_amp', noise_amps],
    ['back_forth', 'n_back_forth', [10]]
    ]

'''another method is to give a list of perturbations to the base config'''
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)
#todo: make metric a list

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=True,
    densify=densify,
    attr_opt=attr_opt,
    metric='auc',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    plot_every= -1
    #! add calculation type and finish the hAk function
    )

In [ ]:
#DDI ieclam AUC
#DECIDE ON A MODEL AND DATASET
model_name = 'ieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS
use_global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.0
n_reps = 1

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following'''
dim_feats = [120, 170]
n_iters_feats = [14000] 
lr_feats = [5*1e-6, 1e-5]
l1_regs = [0.1, 0.5, 0.7]
s_regs = [0.0, 0.1]
range_triplets = [
    # ['clamiter_init','s_reg', s_regs],
    # ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
    ]

# if model_name in ['pclam', 'pieclam']:
#     range_triplets += [
#         ['back_forth', 'first_func_in_fit', first_funcs_in_fit],
#         ['prior_opt','n_iter', n_iters_prior],
#         ['prior_opt','lr', lr_prior],
#         ['prior_opt','noise_amp', noise_amps],
#         ['back_forth','n_back_forth', n_back_forth]
#     ]

'''another method is to give a list of perturbations to the base config'''
#! when i do the deltas the parameters arent saved
# deltas ={
# # 'clamiter_init': {'s_reg': 0.001,
# #                   'l1_reg': 0.001,
# #                   'dim_feat': 2},
# # 'feat_opt': {'n_iter': 50,
# #             'lr': 0.0000001},
# # 'prior_opt': {'n_iter': 50,
# #               'lr': 0.0000001,
# #               'noise_amp': 0.005},
# 'back_forth': {'scheduler_gamma' : 0.1}
# }

# range_triplets = ou.perturb_config('link_prediction',
#                                    ds_name, 
#                                    model_name, 
#                                    deltas, 
#                                    use_global_config=use_global_config_base)

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=use_global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    metric='auc',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    plot_every= 7000
    )
#test that texas still works


In [ ]:
#DDI only on test dataset
#DECIDE ON A MODEL AND DATASET
model_name = 'ieclam'
ds_name = 'ogbl-ddi'

# SET RANGE PARAMETERS

densify = False
attr_opt = False
n_reps = 10

#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=[],
    use_global_config_base=False,
    densify=densify,
    attr_opt=attr_opt,
    metric='auc',
    n_reps=n_reps,
    acc_every=100,
    device=device,
    test_only=True,
    plot_every= -1
    #! add calculation type and finish the hAk function
    )

#todo: check how the run is going. GOING WELL! 
#todo: see that the hyper files are the same as in the paper
#todo: make slurm runs for ogb auc and h@20
#todo: anomaly detection see everything runs
#todo: UPLOAD TO GIT Asap
#todo: test script in jupyter notebook
#test that texas still works

## Texas

In [ ]:
#TEXAS crossval
#DECIDE ON A MODEL AND DATASET


# First find parameters on the validation set and use these parameters to train on the training U validation set

model_name = 'pieclam'
ds_name = 'texas'


# SET RANGE PARAMETERS
use_global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.05
n_reps = 10

'''to do cross validation, give a list of parameters for every one of the parameters as given in the following. There is also an option to use the global parameter configuration as shown below. each field is a list of values for that variable'''
# range_triplets = [
#     ['clamiter_init','s_reg', s_regs],
#     ['clamiter_init','l1_reg', l1_regs],
#     ['clamiter_init', 'dim_feat', dim_feats],
#     ['feat_opt','n_iter', n_iters_feats],
#     ['feat_opt','lr', lr_feats],
# ]

# if model_name in ['pclam', 'pieclam']:
#     range_triplets += [
#         ['back_forth', 'first_func_in_fit', first_funcs_in_fit],
#         ['prior_opt','n_iter', n_iters_prior],
#         ['prior_opt','lr', lr_prior],
#         ['prior_opt','noise_amp', noise_amps],
#         ['back_forth','n_back_forth', n_back_forth]
#     ]

'''another method is to give a list of perturbations to the base config from which you get the range triplets for the cross val'''

deltas ={
'clamiter_init': {'s_reg': 0.001,
                  'l1_reg': 0.001,
                  'dim_feat': 2},
'feat_opt': {'n_iter': 50,
            'lr': 0.0000001},
'prior_opt': {'n_iter': 50,
              'lr': 0.0000001,
              'noise_amp': 0.005},
'back_forth': {'scheduler_gamma' : 0.1}
}

range_triplets = ou.perturb_config('anomaly_unsupervised', 
                                   model_name, 
                                   deltas, 
                                   use_global_config=use_global_config_base)

range_triplets += [
                    ['back_forth', 'n_back_forth',[1]]
                ]


#RUN CROSS VALIDATION ON THE DATASET WITH THE MODEL
# accuracy change
# 
#todo: ACCURACY: add metric variable and collect test and val values according to it

#todo: if it's a validation task save to validation folder if it's a test task save to test folder
#todo: add the metric folder split. /results/task/data/model/metric/test or val

#! this needs to be changed in anomaly detection as well!
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    use_global_config_base=use_global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    metric='auc',
    test_p=test_p,
    val_p=0.0,
    n_reps=n_reps,
    device=device,
    verbose=True,
    verbose_in_funcs=False
    )
